In [2]:
import pandas
import numpy

data = pandas.read_csv("WineData.csv")
print(data.iloc[1:10])

   fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   
5            7.4              0.66         0.00             1.8      0.075   
6            7.9              0.60         0.06             1.6      0.069   
7            7.3              0.65         0.00             1.2      0.065   
8            7.8              0.58         0.02             2.0      0.073   
9            7.5              0.50         0.36             6.1      0.071   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65 

In [3]:
data.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
color                    object
dtype: object

In [4]:
data['color'] = data['color'].replace(to_replace = 'white' , value = '0')
data['color'] = data['color'].replace(to_replace = 'red' , value = '1')

In [5]:
# All data columns except for color
feature_cols = [x for x in data.columns if x not in 'color']


In [6]:

from sklearn.model_selection import StratifiedShuffleSplit

# Split the data into two parts with 1000 points in the test data
# This creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=1000, random_state=42)


In [7]:

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['color']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'color']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'color']

In [8]:
# Use o método value_counts com normalização para verificar a distribuição
y_train.value_counts(normalize=True)

0    0.753866
1    0.246134
Name: color, dtype: float64

In [9]:
y_test.value_counts(normalize=True)

0    0.754
1    0.246
Name: color, dtype: float64

In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train,y_train)

In [26]:
# quantos nós e profundidade
#[estimator.tree_.max_depth for estimator in dt.estimators_]
print(dt.tree_.max_depth)
#print(dt.tree_.nodes)

22


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# função para retornar diversas métricas
def measure_error(y_true, y_pred, label):
    return pandas.Series({'accuracy':accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)


In [23]:
# The error on the training and test data sets
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

#train_test_full_error = pandas.concat([measure_error(y_train, y_train_pred, 'train'),
                                       #measure_error(y_test, y_test_pred, 'test')],axis = 1)

#train_test_full_error

In [15]:
from sklearn.model_selection import GridSearchCV

# variação de parâmetros que queremos testar
param_grid = {'max_depth':range(1, dt.tree_.max_depth+1, 2),
              'max_features': range(1, len(dt.feature_importances_)+1)}

GR = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  param_grid=param_grid,
                  scoring='accuracy',  # vamos maximizar acurácia, teste com outras métricas
                  n_jobs=-1)

# aplicamos apenas na base de treino, obviamente
GR = GR.fit(X_train,y_train)
print(GR)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': range(1, 13), 'max_depth': range(1, 23, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)


In [30]:
# número de nós e profundidade
GR.best_estimator_.tree_.max_depth
#GR.best_estimator_.tree_.nodes??
#In [ ]:
#y_train_pred_gr = GR.predict(???)
#y_test_pred_gr = GR.predict(???)

#train_test_gr_error = pd.concat([measure_error(y_train, y_train_pred_gr, 'train'),
#                                 measure_error(y_test, y_test_pred_gr, 'test')],
#                                axis=1)

7